In [191]:
import base64
import IPython
import os
import uuid
import glob
import re

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import Dropout
from keras.preprocessing import sequence

In [40]:
os.chdir("c:/numbers_recognition/")

In [34]:
import librosa

import numpy as np

import librosa.core
import librosa.display
import librosa.effects

In [188]:
import keras
import keras.utils
import random
import numpy.random
import pickle

In [149]:
import keras.preprocessing.text

In [26]:
def ds_complete(file):
    label, gender, media = [], [], []
    with open(file, "r") as f:
        for line in f:
            l,g,m = line.split(',')
            label.append(l.strip())
            gender.append(g.strip())
            media.append(m.strip())
    return label, gender, media

In [178]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras Sequence dataset'
    
    def __init__(self, files, labels, path="wav", samples=10, batch_size=16, maxlen=5e5, n_classes=30, shuffle=True):
        'Initialize class'
        
        self.batch_size = batch_size
        
        # labels
        self.labels = labels
        
        # files
        self.files = files
        self.csvs = [os.path.join(path, re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", x)[0]+".wav") for x in files]
        self.uuids = [re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", x)[0] for x in files]
        self.label_by_file = dict(zip(files, labels))
        self.label_by_uuid = dict(zip(self.uuids, labels))
        
        # params
        self.path = path
        self.maxlen = int(maxlen)
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.samples = samples
        

    def __len__(self):
        'Returns the number of batches per epoch'
        return self.samples
    

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        files = [self.files[random.randint(0, len(self.files)-1)] for x in range(self.batch_size)]
        X, y = self.__data_generation(files)

        return X, y


    def __data_generation(self, files):
        'Generates data containing batch_size samples' 
        
        X = np.empty((self.batch_size, self.maxlen))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, file in enumerate(files):
            # load file
            # MONO rate=8k auto trimming
            fn = os.path.join(self.path, re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", file)[0]+".wav")
            data, fs = librosa.load(fn, mono=True, sr=8e3)
            trimed, index = librosa.effects.trim(data, top_db=29, frame_length=10)
            
            if self.shuffle:
                rs = random.randint(100,1000)
                rss = random.randint(5000,10000)

                smp = np.concatenate( (np.random.ranf(size=rs)/1e3 , trimed[rs:], np.random.ranf(size=rss)/1e3), axis=0)
            else:
                smp = trimed
            
            X[i,] = np.concatenate( (smp[: min(self.maxlen, len(smp))], np.zeros(( self.maxlen - min(self.maxlen, len(smp))))), axis=0)

            # Store class
            y[i] = tokenizer.word_index[str(self.label_by_file[file])]
            # print(y)

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes) # Why? need + 1 ???

In [41]:
csv = glob.glob("dataset/*.completed.csv");

In [150]:
tokenizer = keras.preprocessing.text.Tokenizer()

In [42]:
file_to_class = {}
uuid_to_class = {}
file_to_gender = {}
uuid_to_gender = {}
files = []
labels = []
gender = []

for f in csv:
    l,g,m = ds_complete(f)
    files += m
    labels += l
    gender += g
    file_to_class.update({"csv/"+re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", x)[0]+".csv":l[i] for i,x in enumerate(m)})
    uuid_to_class.update({re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", x)[0]:l[i] for i,x in enumerate(m)})
    file_to_gender.update({"csv/"+re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", x)[0]+".csv":g[i] for i,x in enumerate(m)})
    uuid_to_gender.update({re.findall("(\w+-\w+-\w+-\w+-\w+)\.dat", x)[0]:g[i] for i,x in enumerate(m)})

In [190]:
uuid_to_class['058f821c-d87b-11e8-997b-f48c5031df2f']

'1'

In [151]:
tokenizer.fit_on_texts(labels)

In [43]:
len(files), len(labels)

(45, 45)

In [179]:
classes=len({x:x for x in labels})
classes = classes + 1

In [193]:
max_review_length = int(1e4)
embedding=100

In [194]:
train = DataGenerator(files=files, labels=labels, samples=100, maxlen=max_review_length, batch_size=16, shuffle=True, n_classes=classes)
test = DataGenerator(files=files, labels=labels, samples=10, maxlen=max_review_length, batch_size=16, shuffle=False, n_classes=classes)

In [195]:
model = Sequential()
# Embedding
model.add(Embedding(max_review_length, embedding, input_length=max_review_length))

# Convilution
model.add(Conv1D(filters=125, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Dropout
model.add(Dropout(0.2))

# LSTM
model.add(LSTM(100))

# Classification
model.add(Dense(classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 10000, 100)        1000000   
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 10000, 125)        62625     
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 5000, 125)         0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 5000, 32)          12032     
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 2500, 32)          0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 2500, 16)          1552      
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 1250, 16)          0         
__________

In [196]:
model.fit_generator(generator=train, steps_per_epoch=50, epochs=5, validation_data=test)

Epoch 1/5
50/50 [==============================] - 144s 3s/step - loss: 0.3098 - acc: 0.9299 - val_loss: 0.1834 - val_acc: 0.9545
Epoch 2/5
50/50 [==============================] - 140s 3s/step - loss: 0.1792 - acc: 0.9545 - val_loss: 0.1769 - val_acc: 0.9545
Epoch 3/5
50/50 [==============================] - 146s 3s/step - loss: 0.1786 - acc: 0.9545 - val_loss: 0.1773 - val_acc: 0.9545
Epoch 4/5
50/50 [==============================] - 140s 3s/step - loss: 0.1794 - acc: 0.9545 - val_loss: 0.1784 - val_acc: 0.9545
Epoch 5/5
50/50 [==============================] - 144s 3s/step - loss: 0.1796 - acc: 0.9545 - val_loss: 0.1786 - val_acc: 0.9545


In [197]:
model.save("nr_5e_22c.h5")

In [189]:
pickle.dump(tokenizer, open("tokenizer_22.pkl", "wb"))